In [ ]:
import instaloader
import time
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, CallbackQueryHandler, MessageHandler, filters
import pandas as pd
import re
import nest_asyncio

# Применяем патч для работы с текущим event loop
nest_asyncio.apply()

# Инициализация Instaloader для анализа профиля
L = instaloader.Instaloader()

# Заменить на свои учетные данные Instagram
L.login("USER_NAME", "PASSWORD")

# Функция для создания таблиц с анализом
async def analyze_instagram_profile(username, update):
    try:
        await update.message.reply_text('Ищу профиль в Instagram...')
        profile = instaloader.Profile.from_username(L.context, username)

        # Получаем основную информацию о профиле
        await update.message.reply_text('Собираю основную информацию о профиле...')
        general_info = {
            "Profile URL": [f"https://instagram.com/{username}"],
            "Posts": [profile.mediacount],
            "Followers": [profile.followers],
            "Followings": [profile.followees],
            "Bio": [profile.biography]
        }

        # Преобразуем итератор постов в список
        await update.message.reply_text('Собираю посты...')
        posts = list(profile.get_posts())

        # Поиск поста с наибольшим количеством лайков
        await update.message.reply_text('Ищу лучший пост по количеству лайков...')
        best_post = max(posts, key=lambda post: post.likes)
        general_info["Best Post URL"] = [f"https://instagram.com/p/{best_post.shortcode}/"]
        general_info["Best Post Likes"] = [best_post.likes]
        general_info["Best Post Comments"] = [best_post.comments]

        # Рассчитываем частоту публикаций
        await update.message.reply_text('Рассчитываю частоту публикаций...')
        if len(posts) > 1:
            post_frequency = (posts[0].date - posts[-1].date).days / len(posts)
        else:
            post_frequency = "N/A"
        
        general_info["Post Frequency (days between posts)"] = [post_frequency]

        # Рассчитываем уровень вовлеченности
        await update.message.reply_text('Рассчитываю вовлечённость...')
        general_info["Engagement Rate"] = [best_post.likes / profile.followers * 100] if profile.followers > 0 else 0

        # Создаем первую таблицу с общей информацией
        await update.message.reply_text('Создаю таблицу с общей информацией...')
        df_general = pd.DataFrame(general_info)
        general_info_file = f"{username}_general_info.xlsx"
        df_general.to_excel(general_info_file, index=False)

        # Отправляем первую таблицу пользователю
        await update.message.reply_text('Отправляю первую таблицу с общей информацией...')
        await update.message.reply_document(document=open(general_info_file, 'rb'))

        # После этого переходим к сбору информации о постах
        await update.message.reply_text('Анализирую посты...')

        # Анализируем 10 постов с паузами между запросами
        post_data = []
        for post in posts[:10]:
            try:
                # Преобразуем комментарии в список, чтобы избежать ошибок с итератором
                comments = list(post.get_comments())
                hashtags = " ".join(post.caption_hashtags)
                top_comments = [f"@{comment.owner.username}: {comment.text}" for comment in comments[:10]]
                external_links = ', '.join([link for link in post.caption_mentions])
                
                # Форматируем дату и время
                post_date = post.date.strftime("%Y-%m-%d")
                post_time = post.date.strftime("%H:%M:%S")
                
                post_data.append({
                    "Post URL": f"https://instagram.com/p/{post.shortcode}/",
                    "Post Date": post_date,
                    "Post Time": post_time,
                    "Likes": post.likes,
                    "Comments": post.comments,
                    "Hashtags": hashtags,
                    "Post Text": post.caption,
                    "Top 10 Comments": "; ".join(top_comments),
                    "External Links": external_links
                })
                # Добавляем паузу между запросами, чтобы избежать блокировок
                time.sleep(2)
            except Exception as e:
                print(f"Error during post analysis: {str(e)}")
                continue

        # Создаем вторую таблицу с анализом постов
        await update.message.reply_text('Создаю таблицу с анализом постов...')
        df_posts = pd.DataFrame(post_data)
        posts_info_file = f"{username}_posts_info.xlsx"
        df_posts.to_excel(posts_info_file, index=False)

        return general_info_file, posts_info_file
    except Exception as e:
        await update.message.reply_text(f'Произошла ошибка: {str(e)}')
        return None, str(e)

# Функция для извлечения username из URL
def extract_username_from_url(profile_url):
    match = re.match(r"(?:http[s]?://)?(?:www\.)?instagram\.com/([A-Za-z0-9._]+)/?", profile_url)
    if match:
        return match.group(1)
    return None

# Хэндлер для старта
async def start(update: Update, context):
    keyboard = [
        [InlineKeyboardButton("Info", callback_data='info')],
        [InlineKeyboardButton("Analysis (table)", callback_data='analysis_table')],
        [InlineKeyboardButton("Analysis (ChatGPT)", callback_data='analysis_chatgpt')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text('Welcome to the Instagram Analysis Bot!', reply_markup=reply_markup)

# Хэндлер для обработки кнопок
async def button_handler(update: Update, context):
    query = update.callback_query
    await query.answer()

    if query.data == 'info':
        await query.message.reply_text('Привет! Этот бот анализирует Instagram профили.')
    elif query.data == 'analysis_table':
        await query.message.reply_text('Пожалуйста, вставьте ссылку на профиль в Instagram (например: https://instagram.com/user_name).')
        context.user_data['action'] = 'analysis_table'

# Хэндлер для обработки сообщений
async def message_handler(update: Update, context):
    if 'action' in context.user_data and context.user_data['action'] == 'analysis_table':
        profile_url = update.message.text.strip()

        # Извлекаем username из URL
        username = extract_username_from_url(profile_url)

        if username:
            await update.message.reply_text(f'Начинаю анализ профиля {username}...')
            general_info_file, posts_info_file = await analyze_instagram_profile(username, update)

            if general_info_file:
                await update.message.reply_document(document=open(posts_info_file, 'rb'))
            else:
                await update.message.reply_text(f'Произошла ошибка: {posts_info_file}')
        else:
            await update.message.reply_text('Неверный формат ссылки. Пожалуйста, вставьте правильную ссылку на профиль Instagram.')

        context.user_data['action'] = None

# Основная функция
if __name__ == '__main__':
    application = ApplicationBuilder().token('TOKEN_TELEGA_BOT').build()

    application.add_handler(CommandHandler('start', start))
    application.add_handler(CallbackQueryHandler(button_handler))
    application.add_handler(MessageHandler(filters.TEXT, message_handler))

    application.run_polling()
